In [1]:
import pandas as pd
import numpy as np
import ta

In [2]:
def clean_ds (df):
    for i in range(1, 4):  
        df[f'X_t-{i}'] = df['Close'].shift(i)

    # Shift Close Column up by 5 rows
    df['Pt_5'] = df['Close'].shift(-5)

    #Agregamos RSI
    rsi_data = ta.momentum.RSIIndicator(close= df['Close'], window=28)
    df['RSI'] = rsi_data.rsi()

    # La Y
    df['Y_BUY'] = df['Close'] * (1 + 0.02) < df['Pt_5']
    df['Y_SELL'] = df['Close'] * (1 - 0.02) > df['Pt_5']
    
    df['Y_BUY'] = df['Y_BUY'].astype(int)
    df['Y_SELL'] = df['Y_SELL'].astype(int)

    return df

In [3]:
data_1 = pd.read_csv('../data/aapl_1d_train.csv')
data_1

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2000-01-03,104.8768,112.5040,101.6848,111.9328,0.8472,535796800
1,2000-01-04,108.2480,110.6224,101.1920,102.5024,0.7758,512377600
2,2000-01-05,103.7456,110.5664,102.9952,104.0032,0.7872,778321600
3,2000-01-06,106.1200,107.0048,94.9984,94.9984,0.7190,767972800
4,2000-01-07,96.4992,101.0016,95.5024,99.5008,0.7531,460734400
...,...,...,...,...,...,...,...
5279,2020-12-24,131.3200,133.4600,131.1000,131.9700,129.6797,54930100
5280,2020-12-28,133.9900,137.3400,133.5100,136.6900,134.3178,124486200
5281,2020-12-29,138.0500,138.7900,134.3400,134.8700,132.5294,121047300
5282,2020-12-30,135.5800,135.9900,133.4000,133.7200,131.3994,96452100


In [6]:
clean_ds(data_1).head(30)

,Date,Open,High,Low,Close,Adjusted_close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,2000-01-03,104.8768,112.5040,101.6848,111.9328,0.8472,535796800,NaN,NaN,NaN,97.7536,NaN,0,1
1,2000-01-04,108.2480,110.6224,101.1920,102.5024,0.7758,512377600,111.9328,NaN,NaN,92.7472,NaN,0,1
2,2000-01-05,103.7456,110.5664,102.9952,104.0032,0.7872,778321600,102.5024,111.9328,NaN,87.1920,NaN,0,1
3,2000-01-06,106.1200,107.0048,94.9984,94.9984,0.7190,767972800,104.0032,102.5024,111.9328,96.7456,NaN,0,0
4,2000-01-07,96.4992,101.0016,95.5024,99.5008,0.7531,460734400,94.9984,104.0032,102.5024,100.4416,NaN,0,0
5,2000-01-10,101.9984,102.2448,94.7520,97.7536,0.7399,505064000,99.5008,94.9984,104.0032,103.9360,NaN,1,0
6,2000-01-11,95.9392,99.3776,90.4960,92.7472,0.7020,441548800,97.7536,99.5008,94.9984,106.5680,NaN,1,0
7,2000-01-12,94.9984,95.5024,86.4976,87.1920,0.6599,976068800,92.7472,97.7536,99.5008,113.5008,NaN,1,0
8,2000-01-13,94.4832,98.7504,92.5008,96.7456,0.7322,1032684800,87.1920,92.7472,97.7536,111.3168,NaN,1,0
9,2000-01-14,100.0048,102.2448,99.3776,100.4416,0.7602,390376000,96.7456,87.1920,92.7472,106.2544,NaN,1,0
